# Notebook 2 – Wrappers Gym, Sauvegarde/Chargement, Multiprocessing, Callbacks et Environnements Personnalisés

Ce notebook propose un survol des fonctionnalités avancées de la librairie [Stable Baselines3](https://github.com/DLR-RM/stable-baselines3) :
1. **Utilisation de wrappers Gym** (limitation du nombre d’étapes, normalisation des actions, etc.)
2. **Sauvegarde et chargement de modèles**
3. **Multiprocessing** et environnements vectorisés
4. **Callbacks** : enregistrement automatique, traçage en temps réel, etc.
5. **Création d’un environnement Gym personnalisé**

Nous utilisons un environnement sous Windows, donc nous évitons certaines dépendances (`xvfb`, `freeglut3-dev`) et nous n’utilisons pas de commandes `apt-get`.


## Installation et imports essentiels
Dans un environnement Python classique, on installera Stable Baselines3 (et les dépendances gym) via :
```
pip install stable-baselines3[extra]
```
ou bien :
```
%pip install "stable-baselines3[extra]>=2.0.0a4"
```
si vous utilisez un notebook. 

Ensuite, on importe les principales classes dont on aura besoin.

In [ ]:
%pip install "stable-baselines3[extra]>=2.0.0a4"  

import os
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor


## 1) Wrappers Gym
Les **Gym wrappers** permettent d’ajouter des transformations aux environnements (limiter la durée d’un épisode, normaliser les actions, etc.). Ci-dessous un exemple très condensé :

**Autres wrappers utiles**  
- `Monitor`: Enregistre automatiquement la récompense et la durée de chaque épisode, ce qui facilite l’analyse.  
- `ClipAction`: S’assure que l’action est bien bornée à `[low, high]`.  
- `FlattenObservation`: Convertit une observation complexe (dict, tuple, etc.) en un simple vecteur Numpy.

*Exemple* : 
```python
import gymnasium as gym
from stable_baselines3.common.monitor import Monitor

env = gym.make('CartPole-v1')
env = Monitor(env)  # suivi auto des rewards, durées d’épisodes
```



In [ ]:
import gymnasium as gym
from gymnasium import spaces

class LimitEpisodeSteps(gym.Wrapper):
    def __init__(self, env, max_steps=100):
        super().__init__(env)
        self.max_steps = max_steps
        self.current_step = 0

    def reset(self, **kwargs):
        self.current_step = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.current_step += 1
        if self.current_step >= self.max_steps:
            truncated = True
        return obs, reward, terminated, truncated, info

# Exemple d'utilisation :
base_env = gym.make("CartPole-v1", render_mode="rgb_array")
wrapped_env = LimitEpisodeSteps(base_env, max_steps=50)

obs, _ = wrapped_env.reset()
done = False
while not done:
    action = wrapped_env.action_space.sample()
    obs, reward, terminated, truncated, _ = wrapped_env.step(action)
    done = terminated or truncated


## 2) Sauvegarde et chargement de modèles
Chaque algorithme de Stable-Baselines3 dispose de méthodes `.save(path)` et `.load(path)`. On peut ainsi conserver un modèle partiellement entraîné ou final, et le recharger pour continuer l’apprentissage ou faire de l’inférence.


In [ ]:
# Entraînement basique sur CartPole
env = gym.make("CartPole-v1")
model = PPO("MlpPolicy", env, verbose=0)
model.learn(5000)

# Sauvegarde
model.save("ppo_cartpole")
del model  # on supprime le modèle de la mémoire

# Rechargement
model = PPO.load("ppo_cartpole", env=env)  # on précise l'env si on veut continuer
# Test rapide
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Reprise du modèle chargé : récompense moyenne={mean_reward:.2f}")


**Attention**  
- `model.save(path)` enregistre uniquement les poids du réseau et l’architecture.  
- Pour **off-policy** (DQN, SAC, TD3...), si vous voulez sauvegarder **la replay buffer** (mémoire d’expériences), il faut utiliser en plus `model.save_replay_buffer(path_replay)`.  
- Cela peut s’avérer lourd en mémoire : prenez garde à la taille de `buffer_size` et à l’espace disque.


## 3) Multiprocessing
Pour accélérer l’apprentissage ou pour avoir une meilleure exploration, on peut exécuter plusieurs environnements en parallèle. Cela se fait via `DummyVecEnv` (qui reste sur un seul process) ou `SubprocVecEnv` (plusieurs processus). Souvent, `DummyVecEnv` est plus rapide pour un petit nombre d’environnements, car la communication inter-processus coûte cher.

On définit une fonction `make_env(env_id, rank, seed=0)` qui crée un environnement, puis on construit un `SubprocVecEnv` (ou un `DummyVecEnv`) :

In [ ]:
def make_env(env_id, rank, seed=0):
    def _init():
        env = gym.make(env_id)
        env.reset(seed=seed + rank)
        return env
    return _init

from stable_baselines3.common.utils import set_random_seed

# Exemple : 4 environnements en parallèle
n_procs = 4
env_id = "CartPole-v1"

vec_env = SubprocVecEnv([make_env(env_id, i) for i in range(n_procs)])

model_mp = A2C("MlpPolicy", vec_env, verbose=0)
model_mp.learn(5000)

# Évaluation finale sur un seul env
test_env = gym.make(env_id)
mean_reward, _ = evaluate_policy(model_mp, test_env, n_eval_episodes=10)
print("Récompense moyenne sur 10 épisodes:", mean_reward)

## 4) Callbacks
Les **callbacks** permettent d’intervenir pendant l’entraînement (pour sauvegarder, tracer en temps réel, etc.). Ils héritent de `BaseCallback`. Quelques exemples :

**Callbacks fournis par Stable-Baselines3**  
- `EvalCallback`: évalue régulièrement le modèle sur un environnement de test (distinct de l’env d’entraînement).  
- `CheckpointCallback`: sauvegarde périodiquement le modèle.  
- `StopTrainingOnRewardThreshold`: arrête l’apprentissage si une récompense cible est atteinte.  

*Tip* : En combinant `EvalCallback` et `CheckpointCallback`, vous pouvez automatiquement enregistrer le “meilleur” modèle selon une métrique d’évaluation.


In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class SimpleCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self._called_once = False

    def _on_step(self) -> bool:
        if not self._called_once:
            print("Callback: Première fois !")
            self._called_once = True
            return True
        print("Callback: Deuxième fois, on arrête l'entraînement.")
        return False  # on interrompt l'apprentissage

# Exemple d'utilisation
model_cb = SAC("MlpPolicy", "Pendulum-v1", verbose=0)
model_cb.learn(total_timesteps=2000, callback=SimpleCallback())


### Exemple de callback pour sauvegarder le meilleur modèle
On peut observer la récompense d’entraînement (monitor) et sauvegarder le modèle lorsqu’on obtient une récompense moyenne record. (Pour un usage plus robuste, on conseille d’utiliser un environnement d’évaluation séparé.)

In [ ]:
import numpy as np
from stable_baselines3.common.results_plotter import load_results, ts2xy

class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, check_freq, log_dir, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Timestep: {self.num_timesteps}")
                    print(
                        f"Meilleur reward: {self.best_mean_reward:.2f}  |  Derniers 100 épisodes: {mean_reward:.2f}"
                    )
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    if self.verbose > 0:
                        print("Nouveau meilleur modèle! Sauvegarde...")
                    self.model.save(self.save_path)
        return True

# Exemple d'utilisation
log_dir = "./logs/"  # dossier de logs
os.makedirs(log_dir, exist_ok=True)
env = gym.make("CartPole-v1")
env = Monitor(env, log_dir)

model_saver = A2C("MlpPolicy", env, verbose=0)
callback_saver = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

model_saver.learn(total_timesteps=5000, callback=callback_saver)

## 5) Créer un environnement Gym personnalisé
Enfin, voici un **exemple minimal** d’environnement Gym personnalisé. Il faut définir :

- `__init__`: définit `self.observation_space` et `self.action_space`.  
- `reset()`: renvoie `(obs, info)` où `obs` ∈ `observation_space`.  
- `step(action)`: renvoie `(obs, reward, terminated, truncated, info)`.  
- Assurez-vous que `obs` respecte la forme indiquée par `observation_space`.  

*Note* : Dans Gym 0.26+ et Gymnasium, on a deux indicateurs de fin : `terminated` et `truncated`.  
- `terminated`: la tâche est terminée parce qu’on est allé au bout (victoire/défaite).  
- `truncated`: la tâche s’arrête par limite de temps ou autre contrainte.


In [ ]:
from gymnasium import spaces

class MyCustomEnv(gym.Env):
    def __init__(self, grid_size=5):
        super().__init__()
        self.grid_size = grid_size
        # On définit l'action_space et l'observation_space
        self.action_space = spaces.Discrete(2)  # ex: 0 = gauche, 1 = droite
        self.observation_space = spaces.Box(low=0, high=self.grid_size, shape=(1,), dtype=np.float32)
        self.agent_pos = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)
        self.agent_pos = np.random.randint(low=0, high=self.grid_size)
        return np.array([self.agent_pos], dtype=np.float32), {}

    def step(self, action):
        if action == 0:  # gauche
            self.agent_pos -= 1
        else:            # droite
            self.agent_pos += 1
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

        reward = 1.0 if self.agent_pos == 0 else 0.0  # ex : on favorise d'aller à 0
        terminated = bool(self.agent_pos == 0)
        truncated = False
        info = {}
        return np.array([self.agent_pos], dtype=np.float32), reward, terminated, truncated, info

    def render(self):
        pass  # Optionnel

    def close(self):
        pass

# Validation
from stable_baselines3.common.env_checker import check_env
env_custom = MyCustomEnv()
check_env(env_custom, warn=True)

# Test rapide
model_custom = PPO("MlpPolicy", env_custom, verbose=0)
model_custom.learn(2000)
mean_reward, _ = evaluate_policy(model_custom, env_custom, n_eval_episodes=10)
print("Récompense moyenne :", mean_reward)

# Conclusion
Dans ce second notebook, nous avons parcouru :

- L’usage de **wrappers Gym** pour modifier un environnement (limiter la durée, normaliser, etc.).
- Les **fonctions de sauvegarde/chargement** de modèles (`.save()` / `.load()`).
- Le **multiprocessing** via `SubprocVecEnv` ou `DummyVecEnv` pour accélérer (ou diversifier) l’apprentissage.
- Les **callbacks**, permettant d’intervenir pendant l’entraînement (sauvegarde automatique du meilleur modèle, monitoring, etc.).
- La **création d’un environnement Gym personnalisé**, validé ensuite par la fonction `check_env` et compatible avec tout algorithme Stable-Baselines3.

Vous pouvez maintenant adapter et combiner ces techniques pour vos propres projets d’Apprentissage par Renforcement !